In [2]:
import math
import pandas as pd
import requests
from urllib.parse import urlencode
import os
import numpy as np
from catboost import CatBoostRanker, Pool
from functools import partial
import json
import pyarrow.parquet as pq
import time
import polars as pl
import fastparquet

In [3]:
import os
os.getcwd()

'/home/admin/train_data'

In [4]:

automarkup = pl.read_parquet(
    'automarkup.parquet',
)

In [5]:
automarkup = automarkup.select('query','video_id','position')

In [6]:

features = pl.read_parquet(
    'features.parquet',
)

In [7]:
videos = pl.read_parquet(
    'videos.parquet',
)

In [8]:
full_df = automarkup.join(
    features,
    how='inner',
    on='video_id',
)
full_df = full_df.join(
    videos,
    how='inner',
    on='video_id',
)
del features
del videos
del automarkup
full_df = full_df.drop('report_date')

In [9]:
full_df = full_df.unique(maintain_order=True)


In [10]:
full_df = full_df.sort('query').with_columns(pl.col('query').rank('min').alias('group_id'))
full_df.head()

query,video_id,position,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days,video_title,channel_title,v_channel_reg_datetime_right,v_channel_type_right,v_category_right,v_pub_datetime_right,group_id
str,str,f64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,"datetime[ns, UTC]",str,str,"datetime[ns, UTC]",u32
""" ""","""video_10854637…",1.0,"""2021-03-18 08:…","""ТВ И СМИ""","""Телепередачи""","""2023-10-28 18:…",2436,2186397,2186397,2186397,136938,4856,102,7039320,0.002221,0.000047,0.000576,0.552711,0.00091,0.002221,0.000047,0.000576,0.552711,0.00091,0.001585,0.000029,0.000438,0.42926,0.000438,"""Экстрасенсы. Б…","""Телеканал ТНТ""",2021-03-18 08:06:30 UTC,"""ТВ И СМИ""","""Телепередачи""",2023-10-28 18:00:13 UTC,1
""" ""","""video_19553340…",5.0,"""2021-03-18 08:…","""ТВ И СМИ""","""Телепередачи""","""2022-09-17 13:…",169,1854712,155512,39826,5698,64,0,5623659,0.000552,0.0,0.000879,0.393085,0.0001,0.000405,0.0,0.000977,0.417158,0.000103,0.000176,0.0,0.000878,0.386276,0.000351,"""Новая Битва эк…","""Телеканал ТНТ""",2021-03-18 08:06:30 UTC,"""ТВ И СМИ""","""Телепередачи""",2022-09-17 13:01:01 UTC,1
""" ""","""video_19553340…",5.0,"""2021-03-18 08:…","""ТВ И СМИ""","""Телепередачи""","""2022-09-17 13:…",194,1993291,138579,26339,3461,120,0,5623659,0.000152,0.0,0.000949,0.398231,0.00019,0.000404,0.0,0.000909,0.401309,0.000159,0.0,0.0,0.000578,0.399884,0.0,"""Новая Битва эк…","""Телеканал ТНТ""",2021-03-18 08:06:30 UTC,"""ТВ И СМИ""","""Телепередачи""",2022-09-17 13:01:01 UTC,1
""" ""","""video_19553340…",5.0,"""2021-03-18 08:…","""ТВ И СМИ""","""Телепередачи""","""2022-09-17 13:…",204,2090350,97059,18558,2533,150,0,5623659,0.000485,0.0,0.001185,0.409257,0.000054,0.000309,0.0,0.000948,0.40837,0.000093,0.001184,0.0,0.001184,0.416502,0.0,"""Новая Битва эк…","""Телеканал ТНТ""",2021-03-18 08:06:30 UTC,"""ТВ И СМИ""","""Телепередачи""",2022-09-17 13:01:01 UTC,1
""" ""","""video_19553340…",5.0,"""2021-03-18 08:…","""ТВ И СМИ""","""Телепередачи""","""2022-09-17 13:…",214,2148966,58616,12067,1686,215,1,5623659,0.001657,0.0,0.00058,0.385017,0.000083,0.001109,0.000017,0.000819,0.390627,0.000171,0.001779,0.0,0.0,0.379004,0.0,"""Новая Битва эк…","""Телеканал ТНТ""",2021-03-18 08:06:30 UTC,"""ТВ И СМИ""","""Телепередачи""",2022-09-17 13:01:01 UTC,1


In [11]:
full_df = full_df.with_columns(pl.col('v_channel_reg_datetime').str.to_datetime().cast(pl.Int64),pl.col('v_pub_datetime').str.to_datetime().cast(pl.Int64))\
    .drop('v_channel_reg_datetime_right','v_category_right','v_pub_datetime_right','v_channel_type_right','video_title')
                         

In [12]:
full_df.head()

query,video_id,position,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days,channel_title,group_id
str,str,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,u32
""" ""","""video_10854637…",1.0,1616054790000000,"""ТВ И СМИ""","""Телепередачи""",1698516013000000,2436,2186397,2186397,2186397,136938,4856,102,7039320,0.002221,0.000047,0.000576,0.552711,0.00091,0.002221,0.000047,0.000576,0.552711,0.00091,0.001585,0.000029,0.000438,0.42926,0.000438,"""Телеканал ТНТ""",1
""" ""","""video_19553340…",5.0,1616054790000000,"""ТВ И СМИ""","""Телепередачи""",1663419661000000,169,1854712,155512,39826,5698,64,0,5623659,0.000552,0.0,0.000879,0.393085,0.0001,0.000405,0.0,0.000977,0.417158,0.000103,0.000176,0.0,0.000878,0.386276,0.000351,"""Телеканал ТНТ""",1
""" ""","""video_19553340…",5.0,1616054790000000,"""ТВ И СМИ""","""Телепередачи""",1663419661000000,194,1993291,138579,26339,3461,120,0,5623659,0.000152,0.0,0.000949,0.398231,0.00019,0.000404,0.0,0.000909,0.401309,0.000159,0.0,0.0,0.000578,0.399884,0.0,"""Телеканал ТНТ""",1
""" ""","""video_19553340…",5.0,1616054790000000,"""ТВ И СМИ""","""Телепередачи""",1663419661000000,204,2090350,97059,18558,2533,150,0,5623659,0.000485,0.0,0.001185,0.409257,0.000054,0.000309,0.0,0.000948,0.40837,0.000093,0.001184,0.0,0.001184,0.416502,0.0,"""Телеканал ТНТ""",1
""" ""","""video_19553340…",5.0,1616054790000000,"""ТВ И СМИ""","""Телепередачи""",1663419661000000,214,2148966,58616,12067,1686,215,1,5623659,0.001657,0.0,0.00058,0.385017,0.000083,0.001109,0.000017,0.000819,0.390627,0.000171,0.001779,0.0,0.0,0.379004,0.0,"""Телеканал ТНТ""",1


In [19]:
full_df = full_df.to_pandas()

In [20]:
import gc
gc.collect()

174

In [21]:
groups = pd.Series(full_df['group_id'].unique())
permutation = groups.sample(frac=1, random_state=42)
train_groups, val_groups, test_groups = np.split(
    permutation,
    [int(0.75 * len(permutation)), int(0.90 * len(permutation))]
)

In [22]:
full_df['target'] =full_df['position']
full_df = full_df.drop('position',axis=1)

In [23]:
train_df = full_df[full_df['group_id'].isin(train_groups)]
val_df = full_df[full_df['group_id'].isin(val_groups)]
test_df = full_df[full_df['group_id'].isin(test_groups)]

In [24]:
train_df = train_df.sort_values('group_id')
val_df = val_df.sort_values('group_id')
test_df = test_df.sort_values('group_id')

In [25]:
metainfo_columns = ['query', 'target','video_id', 'group_id']

X_train = train_df.drop(metainfo_columns, axis=1)
y_train, g_train = train_df['target'], train_df['group_id']

X_val = val_df.drop(metainfo_columns, axis=1)
y_val, g_val = val_df['target'], val_df['group_id']

X_test = test_df.drop(metainfo_columns, axis=1)
y_test, g_test = test_df['target'], test_df['group_id']

In [26]:
train = Pool(
    data=X_train.values,
    label=y_train.values,
    group_id=g_train.values,
    feature_names=X_train.columns.to_list(),
    cat_features=['v_channel_type','v_category','channel_title']
)

val = Pool(
    data=X_val.values,
    label=y_val.values,
    group_id=g_val.values,
    feature_names=X_val.columns.to_list(),
    cat_features=['v_channel_type','v_category','channel_title']

)

test = Pool(
    data=X_test.values,
    label=y_test.values,
    group_id=g_test.values,
    feature_names=X_test.columns.to_list(),
    cat_features=['v_channel_type','v_category','channel_title']

)

# Обучение модели

# Измерение метрик

In [29]:
def _metrics_at(at, model, pool, metric='NDCG'):
    metric = metric + f':top={at}'
    eval_metrics = model.eval_metrics(pool, metrics=[metric])
    best_metrics = {}
    for key in eval_metrics.keys():
        best_metrics[key] = eval_metrics[key][model.best_iteration_]
    return best_metrics



In [30]:
import optuna


def objective(trial):
    param = {
        # "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1), cpu only
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        ),
        "used_ram_limit": "70gb",
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 30, 3000),
        'max_bin': trial.suggest_int('max_bin', 100, 400),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log=True),
        'task_type': 'GPU',
        'eval_metric': 'NDCG',
        'random_seed': 42,
        'loss_function': 'QueryRMSE',
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        

   
    gbm =CatBoostRanker(**param)

    gbm.fit(
        train,
        eval_set=val,
        # plot=True,
        use_best_model=True,
        metric_period=metric_period
    )

    return list(_metrics_at(5,gbm,val).values())[0]


In [31]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=1000, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2023-11-25 10:05:56,762] A new study created in memory with name: no-name-05b162b0-069b-409b-8ddf-f1e5c09d9054


Groupwise loss function. OneHotMaxSize set to 10


Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.9636565	best: 0.9636565 (0)	total: 161ms	remaining: 7m 3s


[W 2023-11-25 10:06:04,975] Trial 0 failed with parameters: {'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'max_depth': 3, 'learning_rate': 0.005, 'n_estimators': 2628, 'max_bin': 328, 'min_data_in_leaf': 214, 'l2_leaf_reg': 0.007988185423836369, 'bagging_temperature': 7.443050838271504} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home/admin/.local/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_19612/1307702511.py", line 33, in objective
    gbm.fit(
  File "/home/admin/.local/lib/python3.8/site-packages/catboost/core.py", line 6099, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, pairs,
  File "/home/admin/.local/lib/python3.8/site-packages/catboost/core.py", line 2319, in _fit
    self._train(
  File "/home/admin/.local/lib/python3.8/site-packages/catboost/core.py", line 1723, in _train
    self._objec

KeyboardInterrupt: 

In [39]:
task_type = 'GPU'
metric_period = 100

parameters = {
        'verbose': False,

    # 'colsample_bylevel': 0.09032942712078151,
 'boosting_type': 'Plain',
 'bootstrap_type': 'Bernoulli',
 'max_depth': 8,
 'learning_rate': 0.08,
 'n_estimators': 50,
 'max_bin': 240,
 'min_data_in_leaf': 88,
 'l2_leaf_reg': 0.6511129757449856,
 'subsample': 0.9552869131128571,
'task_type': 'GPU',
        'eval_metric': 'NDCG',
        'random_seed': 42,
        'loss_function': 'QueryRMSE',
}

        

In [40]:
model = CatBoostRanker(**parameters)
model = model.fit(
    train,
    eval_set=val,
    plot=True,
    use_best_model=True,
    metric_period=metric_period
)
model.save_model('ranker.ckpt')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [41]:
import os
os.getcwd()

'/home/admin/train_data'

In [ ]:


metrics_train_at = partial(
    _metrics_at,
    model=model, 
    pool=train
)

metrics_val_at = partial(
    _metrics_at, 
    model=model, 
    pool=val
)

metrics_test_at = partial(
    _metrics_at, 
    model=model, 
    pool=test
)

In [ ]:
metrics_train_at(5)

In [ ]:
metrics_val_at(5)

In [ ]:
metrics_test_at(5)